<a href="https://colab.research.google.com/github/saurabhIU/Deep-Learning/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Dependencies
import librosa
import librosa.display as disp
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time;

In [62]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test_x_01.wav to test_x_01 (1).wav
Saving test_x_02.wav to test_x_02 (1).wav
Saving train_clean_male.wav to train_clean_male (1).wav
Saving train_dirty_male.wav to train_dirty_male (2).wav
User uploaded file "test_x_01.wav" with length 145284 bytes
User uploaded file "test_x_02.wav" with length 388752 bytes
User uploaded file "train_clean_male.wav" with length 2522886 bytes
User uploaded file "train_dirty_male.wav" with length 2522898 bytes


In [0]:
s_clean, sr_clean=librosa.load('train_clean_male.wav',sr=None)
S=librosa.stft(s_clean, n_fft=1024, hop_length=512)
s_noisy, sr_noisy=librosa.load('train_dirty_male.wav',sr=None)
X=librosa.stft(s_noisy, n_fft=1024, hop_length=512)

In [0]:
S_absolute = np.abs(S)
X_absolute = np.abs(X)

In [0]:
EPOCHS = 1000
X = tf.placeholder("float", [None, 513])
Y = tf.placeholder("float", [None, 513])
def nn_model(x):
  
    layer1 = tf.layers.dense(x, 1024,activation=tf.nn.sigmoid)
    layer2 = tf.layers.dense(layer1, 1024,activation=tf.nn.sigmoid)
    logits = tf.layers.dense(layer2, 513,activation=tf.nn.relu)
    return logits

logits = nn_model(X)
cost = tf.losses.mean_squared_error(Y,logits)
optimizer = tf.train.AdamOptimizer().minimize(cost)
prediction = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(prediction, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [98]:
sess =  tf.Session() 
    
sess.run(tf.global_variables_initializer())
  
tic = time.time()
for i in range(EPOCHS):
    c,_,acc = sess.run([cost, optimizer, accuracy],feed_dict={X: X_absolute.T, Y: S_absolute.T})
    if i % 100 == 0:
      print(f'Epoch: {i},training accuracy:{acc * 100}')
toc = time.time()
print(f'Time taken for training is {toc-tic}')

Epoch: 0,training accuracy:0.08133387309499085
Epoch: 100,training accuracy:18.015453219413757
Epoch: 200,training accuracy:27.97885239124298
Epoch: 300,training accuracy:31.232208013534546
Epoch: 400,training accuracy:32.45221674442291
Epoch: 500,training accuracy:30.82553744316101
Epoch: 600,training accuracy:33.55022370815277
Epoch: 700,training accuracy:32.98088610172272
Epoch: 800,training accuracy:33.91622602939606
Epoch: 900,training accuracy:32.00488090515137
Time taken for training is 19.6909339427948


In [99]:
!ls

 cleaned_test1.wav   'test_x_02 (1).wav'	 'train_dirty_male (1).wav'
 sample_data	      test_x_02.wav		 'train_dirty_male (2).wav'
'test_x_01 (1).wav'  'train_clean_male (1).wav'   train_dirty_male.wav
 test_x_01.wav	      train_clean_male.wav


In [0]:
test1, sr_test=librosa.load('test_x_01.wav',sr=None)
Test1=librosa.stft(test1, n_fft=1024, hop_length=512)
Test1_absolute = np.abs(Test1)

In [101]:
Test1_absolute.shape

(513, 142)

In [0]:
test_prediction = sess.run(prediction,feed_dict={X: Test1_absolute.T})

In [0]:
test1_complex = np.multiply(Test1,test_prediction.T)

In [104]:
test1_complex.shape

(513, 142)

In [0]:
test1_timedomain = librosa.istft(test1_complex,hop_length=512, win_length=1024)

In [0]:
librosa.output.write_wav('cleaned_test1.wav', test1_timedomain, sr_test)

In [107]:
import IPython.display as ipd
ipd.Audio('cleaned_test1.wav')

In [88]:
ipd.Audio('test_x_01.wav')